In [5]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_27.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Apply threshold check and adjust max_corr based on tau direction
    if max_corr >= threshold:
        return max_corr if best_tau >= 0 else -max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


filename_sub_1 = "27_sub_mat_1.csv"
compute_submatrix(data, group_1, filename_sub_1, max_tau)

filename_sub_2 = "27_sub_mat_2.csv"
compute_submatrix(data, group_2, filename_sub_2, max_tau)

filename_sub_3 = "27_sub_mat_3.csv"
compute_submatrix(data, group_3, filename_sub_3, max_tau)

filename_cross_1 = "27_cross_matrix_1_2.csv"
compute_cross_group_matrix(data, group_1, group_2, filename_cross_1)

filename_cross_2 = "27_cross_matrix_1_3.csv"
compute_cross_group_matrix(data, group_1, group_3, filename_cross_2)

filename_cross_3 = "27_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross_3)


Submatrix 27_sub_mat_1.csv has been saved.
Submatrix 27_sub_mat_2.csv has been saved.
Submatrix 27_sub_mat_3.csv has been saved.
submatrix 27_cross_matrix_1_2.csv has been saved
submatrix 27_cross_matrix_1_3.csv has been saved
submatrix 27_cross_matrix_2_3.csv has been saved


38099.0   38101.0   38103.0   38105.0   39001.0   39003.0   39005.0  \
21083.0  0.858706 -0.890443  0.827831  0.896078  0.810626  0.859095    0.8528   
21085.0 -0.922792  -0.94565  0.949176 -0.953316  0.930285  0.948412  0.925684   
21087.0 -0.818339 -0.923283 -0.886742 -0.894965 -0.846104 -0.879437  0.865664   
21089.0 -0.848117 -0.874515  0.897017  0.927497 -0.874044  0.899773  0.921393   
21091.0 -0.854739 -0.882728 -0.893337 -0.900945  0.906331 -0.907498  0.898537   
...           ...       ...       ...       ...       ...       ...       ...   
38089.0  0.978065 -1.010551  1.035512  1.011346  0.985712  1.021047  0.987764   
38091.0  0.673113 -0.736978 -0.701951 -0.761965  0.652702 -0.671201 -0.666659   
38093.0 -1.016272 -0.986731 -0.961587  1.023063 -0.977714 -0.975752  0.996557   
38095.0  0.691136  0.833588  0.735251  0.730979   0.75464  0.759382  0.739364   
38097.0  0.976036  0.966127   1.00559  0.952592  0.965413  0.994623  0.941601   

          39007.0   39009.0   39011.0  ...   56027.0   56029.0   56031.0  \
21083.0  0.687848  0.848933  0.850115  ...  -0.60083 -0.853047 -0.856354   
21085.0  0.772169  0.880821 -0.939837  ... -0.570691 -0.882722 -0.880721   
21087.0  0.757237  0.868847 -0.862507  ... -0.533865 -0.783132 -0.845926   
21089.0  0.753437  0.822925 -0.951402  ... -0.573871 -0.823355 -0.844241   
21091.0  0.753184  0.870443 -0.904135  ... -0.517344 -0.814403 -0.852048   
...           ...       ...       ...  ...       ...       ...       ...   
38089.0 -0.837711  0.960042 -0.980459  ... -0.608303 -0.954435 -0.906556   
38091.0 -0.661451 -0.673041 -0.698993  ... -0.536028 -0.618116 -0.526387   
38093.0 -0.797261 -0.909358  0.996287  ... -0.630554 -0.929004 -0.980225   
38095.0 -0.805943  0.814002 -0.677354  ... -0.784454 -0.695624  0.664112   
38097.0  0.804211  0.926224  0.939041  ... -0.629269 -0.928321  0.859098   

          56033.0   56035.0   56037.0   56039.0   56041.0   56043.0   56045.0  
21083.0  0.763338  0.893653  0.833057 -0.878824 -0.910277  0.694335 -0.814835  
21085.0 -0.687491 -0.981885 -0.802956  -0.88488   -0.9138 -0.607875 -0.859882  
21087.0 -0.608315 -0.906737 -0.703741 -0.760058 -0.847425  0.540306 -0.799096  
21089.0 -0.637718 -0.923498 -0.740028 -0.823612 -0.885172 -0.565152 -0.809666  
21091.0 -0.702273 -0.937322  -0.82678 -0.818101 -0.895064 -0.597895 -0.811347  
...           ...       ...       ...       ...       ...       ...       ...  
38089.0 -0.663469 -0.976748 -0.791279 -0.941635 -0.935214   0.57542 -0.831999  
38091.0 -0.558158  -0.66707 -0.644662 -0.625576 -0.632737  0.426246  0.539386  
38093.0 -0.670707 -0.966789 -0.801197 -0.913269  -0.96115  -0.59475 -0.968893  
38095.0  0.606061 -0.685827  0.674606   0.66533  0.715716  0.558071  0.680716  
38097.0 -0.690942 -0.943602 -0.806847  -0.90598 -0.926288 -0.627356  0.891629  

[1000 rows x 1104 columns]